This notebook reads sentiment for each sentence for Excel, and adds an overall sentiment to each comment in MongoDB.

In [1]:
import pandas as pd
import numpy as np
import re
import pymongo

In [354]:
df = pd.read_excel('../data/ESP.xlsx', sheet_name='esp', header=None)

In [355]:
esp_data = df[[0,1,2]]
esp_data = esp_data.rename(index=str, columns={0: "id", 1: "sentence", 2 : "polarity"})
esp_data.head()

,id,sentence,polarity
0,es_1004_enquestes_professors_1_1,Es muy dura calificando.,NEGATIVE
1,es_100_enquestes_professors_2_1,Creo que debería esmerarse un poco más a la ho...,NEGATIVE
2,es_100_enquestes_professors_2_2,Desmuestra una falta deorganización y seriedad...,NEGATIVE
3,es_1018_enquestes_professors_1_1,Una de las cosas que me han gustado han sido l...,NEGATIVE
4,es_1018_enquestes_professors_1_2,Están muy claras y se entiende claramente lo q...,NEGATIVE


In [71]:
conn=pymongo.MongoClient()
db = conn["db_comments_processed"]

In [356]:
polarities = {}
for row in esp_data.values.tolist():
    text = row[1]
    polarity = row[2]
    polarities[text] = polarity


In [374]:
for elem in db.comments_processed.find({},no_cursor_timeout=True):
    if elem['language'] == 'es':
        p = []
        for sentence in elem['sentences']:
            p.append(polarities[sentence])
        if "NEGATIVE" in p and "POSITIVE" in p:
            if p.count("NEGATIVE") > p.count("POSITIVE"):
                sentiment = "NEGATIVE"
            elif p.count("NEGATIVE")== p.count("POSITIVE"):
                sentiment = "NEUTRAL"
            else:
                sentiment = "POSITIVE"
        elif "NEGATIVE" in p:
            sentiment = "NEGATIVE"
        elif "POSITIVE" in p:
            sentiment = "POSITIVE"
        else:
            sentiment = "NEUTRAL"
        db.comments_processed.update({"_id": elem["_id"]}, {"$set":{"sentiment_polarity": sentiment}})

/Users/indra/anaconda/envs/icutestenv/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


In [25]:
df1 = pd.read_excel('../data/ca_enquestes_assignatures_ALL.xlsx', sheet_name='CAT_assignatures_all_polarity', header=None)
df2 = pd.read_excel('../data/ca_enquestes_professors_ALL.xlsx', sheet_name='cat 1000-final', header=None)

In [26]:
cat1_data = df1[[0,1,2]]
cat1_data = cat1_data.rename(index=str, columns={0: "id", 1: "sentence", 2 : "polarity"})
cat1_data.head()

,id,sentence,polarity
0,ca_1429_enquestes_assignatures_3_1,Classes teòriques on passem 2 hores mirant un ...,NEGATIVE
1,ca_1429_enquestes_assignatures_3_2,Examens amb preguntes tipus 'quin és el robot ...,NEGATIVE
2,ca_1429_enquestes_assignatures_3_3,"Les pràctiques, a excepció de la primera part ...",POSITIVE
3,ca_1429_enquestes_assignatures_3_4,"Teoria superficial, a ratos repetitiva i solap...",NEGATIVE
4,ca_1430_enquestes_assignatures_3_1,Posar les notes de les practiques estaria bast...,NEGATIVE


In [27]:
cat2_data = df2[[0,1,2]]
cat2_data = cat2_data.rename(index=str, columns={0: "id", 1: "sentence", 2 : "polarity"})
cat2_data.head()

,id,sentence,polarity
0,ca_1_enquestes_professors_1_1,Les explicacions en les classes de teoria.,POSITIVE
1,ca_10_enquestes_professors_1_1,Claredat,POSITIVE
2,ca_1000_enquestes_professors_2_1,"Ell no explica, ell arriba a classe i obre un ...",NEGATIVE
3,ca_1000_enquestes_professors_2_2,"Els questionaris estàn mal redactats, les pràc...",NEGATIVE
4,ca_1000_enquestes_professors_2_3,Ja ho he dit a l'enquesta de l'assignatura: aq...,NEGATIVE


In [28]:
conn=pymongo.MongoClient()
db = conn["db_comments_processed"]

In [29]:
polarities = {}
for row in cat1_data.values.tolist():
    text = row[1]
    polarity = row[2]
    polarities[text] = polarity
for row in cat2_data.values.tolist():
    text = row[1]
    polarity = row[2]
    polarities[text] = polarity

In [31]:
for elem in db.comments_processed.find({},no_cursor_timeout=True):
    if elem['language'] == 'ca':
        p = []
        for sentence in elem['sentences']:
            if sentence in polarities:
                p.append(polarities[sentence])
            if "NEGATIVE" in p and "POSITIVE" in p:
                if p.count("NEGATIVE") > p.count("POSITIVE"):
                    sentiment = "NEGATIVE"
                elif p.count("NEGATIVE")== p.count("POSITIVE"):
                    sentiment = "NEUTRAL"
                else:
                    sentiment = "POSITIVE"
            elif "NEGATIVE" in p:
                sentiment = "NEGATIVE"
            elif "POSITIVE" in p:
                sentiment = "POSITIVE"
            else:
                sentiment = "NEUTRAL"
            db.comments_processed.update({"_id": elem["_id"]}, {"$set":{"sentiment_polarity": sentiment}})

/Users/indra/anaconda/envs/icutestenv/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
